In [6]:
import os
import numpy as np
import cv2
#from tensorflow import keras
#from tensorflow.keras import layers
from PIL import Image

In [3]:
def resize_image(input_path, base_width):
    img = Image.open(input_path)
    w_percent = (base_width / float(img.size[0]))
    h_size = int((float(img.size[1]) * float(w_percent)))
    return img.resize((base_width, h_size), Image.ANTIALIAS)

In [65]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import numpy as np

# Define the list of classes
#classes = ['Burlant', 'Buttnera', 'Kordia', 'Rivan', 'Sam', 'Summit', 'Van', 'Vega']
classes = [ 'Rivan', 'Sam', 'Summit']

# Set the directory containing the original images
data_directory = 'data/'

# Create a directory to save augmented images
augmented_directory = 'augmented_data_validation/'
base_width = 800  # The desired width for resizing

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=40,  # Rotate images up to 40 degrees
    width_shift_range=0.25,  # Shift width by up to 20%
    height_shift_range=0.25,  # Shift height by up to 20%
    shear_range=0.2,  # Shear transformations
    zoom_range=0.2,  # Zoom in or out by 20%
    horizontal_flip=True,  # Horizontal flips
    fill_mode='nearest'  # Fill in missing pixels using the nearest available
)

# Loop through each class
for class_name in classes:
    # Define the path to the folder containing images for the current class
    class_folder_path = os.path.join(data_directory, class_name)

    # Create a directory for the current class in the augmented data folder
    class_directory = os.path.join(augmented_directory, class_name)
    os.makedirs(class_directory, exist_ok=True)

    # Loop through each image in the class folder
    for i, image_file in enumerate(os.listdir(class_folder_path)):
        # Check if the file is a valid image file (you may want to add more checks)
        if image_file.lower().endswith(('.bmp', '.jpg', '.jpeg', '.png')):
            # Define the path to the original image
            original_image_path = os.path.join(class_folder_path, image_file)

            # Load the original image
            resized_img = resize_image(original_image_path, base_width)
            x = np.array(resized_img)
            x = x.reshape((1,) + x.shape)  

            # Generate augmented images
            for j, batch in enumerate( datagen.flow(x, batch_size=1), start=1):
                augmented_image = batch[0].astype(np.uint8)
                augmented_image_path = os.path.join(class_directory, f'{class_name}_{i}_{j}.bmp')
                
                Image.fromarray(augmented_image).save(augmented_image_path)
                if j >= 7:  # Generate a desired number of augmented images per class
                    break

KeyboardInterrupt: 

In [ ]:
# count run 1
import os

# Function to count files in a directory
def count_files_in_directory(directory):
    file_count = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_count += 1
    return file_count

# Root folder path
root_folder = 'augmented_data_validation'

# List all subdirectories in the root folder
subfolders = [os.path.join(root_folder, d) for d in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, d))]

# Count and display files in each subfolder
for subfolder in subfolders:
    file_count = count_files_in_directory(subfolder)
    print(f"Folder: {subfolder}, File Count: {file_count}")


Folder: augmented_data_validation\Burlant, File Count: 812
Folder: augmented_data_validation\Buttnera, File Count: 203
Folder: augmented_data_validation\Kordia, File Count: 630
Folder: augmented_data_validation\Rivan, File Count: 119
Folder: augmented_data_validation\Sam, File Count: 98
Folder: augmented_data_validation\Summit, File Count: 112
Folder: augmented_data_validation\Van, File Count: 707
Folder: augmented_data_validation\Vega, File Count: 840


KeyboardInterrupt: 

In [53]:
import os
import shutil
source_directory = "augmented_data_validation"
destination_directory = "augmented_data_balanced"
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)
target_file_count = min([812, 203, 630, 119, 98, 112, 707, 840])
for folder_name in os.listdir(source_directory):
    source_folder = os.path.join(source_directory, folder_name)
    destination_folder = os.path.join(destination_directory, folder_name)

    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # List all files in the source folder
    files = os.listdir(source_folder)

    # Copy the required number of files to the destination folder to balance them
    for i in range(target_file_count):
        file_to_copy = files[i % len(files)]  # Use modulo to loop through files
        source_file_path = os.path.join(source_folder, file_to_copy)
        destination_file_path = os.path.join(destination_folder, file_to_copy)
        shutil.copyfile(source_file_path, destination_file_path)


In [40]:
# import os
# from PIL import Image

# def convert_to_grayscale(input_path, output_path):
#     img = Image.open(input_path)
#     img = img.convert('L')  # Convert to grayscale
#     img.save(output_path)

# def process_directory(input_root, output_root):
#     for subdir, dirs, files in os.walk(input_root):
#         for file in files:
#             if file.lower().endswith(('.bmp', '.jpg', '.jpeg', '.png')):
#                 input_path = os.path.join(subdir, file)
#                 output_subdir = subdir.replace(input_root, output_root, 1)

#                 if not os.path.exists(output_subdir):
#                     os.makedirs(output_subdir)

#                 output_path = os.path.join(output_subdir, file)
#                 convert_to_grayscale(input_path, output_path)

# # Paths to your directories
# input_root = 'augmented_data_validation/'
# output_root = 'augmented_data_gray_scale/'

# process_directory(input_root, output_root)


In [54]:
from sklearn.model_selection import train_test_split
classes = ['Burlant', 'Buttnera', 'Kordia', 'Rivan', 'Sam', 'Summit', 'Van', 'Vega']
augmented_directory = 'augmented_data_balanced\\'

# Initialize lists to hold image paths and corresponding labels
image_paths = []
labels = []

for class_name in classes:
    class_directory = os.path.join(augmented_directory, class_name)
    image_files = os.listdir(class_directory)
    image_paths.extend([os.path.join(class_directory, img) for img in image_files])
    labels.extend([class_name] * len(image_files))

# Split data into training and testing sets
test_size = 0.2  # 20% for testing
X_train, X_test, y_train, y_test = train_test_split(image_paths, labels, test_size=test_size, random_state=42)

In [55]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

def preprocess_image(image_path, target_size):
    img = load_img(image_path, target_size=target_size)
    img = img_to_array(img)
    img /= 255.0  # Normalize pixel values
    return img

target_size = (224, 224)  # Adjust to match your model's input size
X_train = [preprocess_image(img, target_size) for img in X_train]
X_test = [preprocess_image(img, target_size) for img in X_test]

y_train = to_categorical([classes.index(label) for label in y_train], len(classes))
y_test = to_categorical([classes.index(label) for label in y_test], len(classes))

X_train = np.array(X_train)
X_test = np.array(X_test)

In [56]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=target_size + (3,))
#base_model = VGG16(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
                   
for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

# Additional Dense Layers
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(classes), activation='softmax')(x)

# vgghist = model.fit(X_train, y_train, steps_per_epoch = 100, epochs = 10)

num_epochs = 70



model = Model(inputs=base_model.input, outputs=predictions)


# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with data augmentation
model.fit(
    X_train, y_train,
    epochs=num_epochs,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
)

Epoch 1/70
20/20 [==============================] - 11s 455ms/step - loss: 3.2977 - accuracy: 0.1435 - val_loss: 2.0712 - val_accuracy: 0.1911
Epoch 2/70
20/20 [==============================] - 8s 430ms/step - loss: 2.8268 - accuracy: 0.2217 - val_loss: 1.8767 - val_accuracy: 0.2866
Epoch 3/70
20/20 [==============================] - 9s 438ms/step - loss: 2.6332 - accuracy: 0.2217 - val_loss: 1.7101 - val_accuracy: 0.3949
Epoch 4/70
20/20 [==============================] - 9s 431ms/step - loss: 2.2749 - accuracy: 0.2743 - val_loss: 1.5782 - val_accuracy: 0.4459
Epoch 5/70
20/20 [==============================] - 8s 425ms/step - loss: 2.2418 - accuracy: 0.2871 - val_loss: 1.4746 - val_accuracy: 0.4777
Epoch 6/70
20/20 [==============================] - 8s 411ms/step - loss: 2.1216 - accuracy: 0.3158 - val_loss: 1.3589 - val_accuracy: 0.5350
Epoch 7/70
20/20 [==============================] - 8s 418ms/step - loss: 1.9369 - accuracy: 0.3557 - val_loss: 1.2593 - val_accuracy: 0.5414
Epoch

In [57]:
model.save("model/leaf_classifier_smaller_82.h5")

In [34]:
# Assuming you have already trained your model using model.fit()

# Evaluate the model on the test data
evaluation = model.evaluate(X_test, y_test)

# Print the accuracy
accuracy = evaluation[1]  # The second element of the evaluation result is accuracy
print(f"Accuracy on the test data: {accuracy}")

10/10 [==============================] - 3s 310ms/step - loss: 1.0585 - accuracy: 0.6457
Accuracy on the test data: 0.6456953883171082


In [1]:
import os


def resize_bmp(input_path, output_path, base_width):
    img = Image.open(input_path)
    w_percent = (base_width / float(img.size[0]))
    h_size = int((float(img.size[1]) * float(w_percent)))
    img = img.resize((base_width, h_size), Image.ANTIALIAS)
    img.save(output_path)

def process_directory(root_dir, output_root, base_width):
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            # Check if the file is an image
            if file.lower().endswith('.bmp'):
                input_path = os.path.join(subdir, file)
                output_subdir = subdir.replace(root_dir, output_root, 1)
                if not os.path.exists(output_subdir):
                    os.makedirs(output_subdir)
                output_path = os.path.join(output_subdir, file)
                resize_bmp(input_path, output_path, base_width)

# Example usage
root_dir = 'augmented_data'
output_root = 'augmented_data_smaller'
base_width = 800
process_directory(root_dir, output_root, base_width)
